# 深層学習ノートブック-4 ミニバッチ勾配降下法
DL_notebook-3_Multinomial_Logistic_Regression.ipynbのコードをベースに  
ミニバッチ勾配降下法を用いてMNISTを学習させる。  

# ●前処理・初期化

In [21]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

In [22]:
# 変数定義
learning_rate = 0.03
loss_log = []  #損失記録用のリスト

In [25]:
# データロード
dataset = datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32)
target = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of y_true: {target.shape}')
print(target)

shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.])
shape of y_true: torch.Size([1797])
tensor([0, 1, 2,  ..., 8, 9, 8])


In [26]:
# 目的変数のエンコーディング
y_true = F.one_hot(target, num_classes=10)
print(f'shape of y_true: {y_true.shape}')

shape of y_true: torch.Size([1797, 10])


In [27]:
# 学習データの標準化（ピクセル値を標準化）
X = torch.nan_to_num((X - X.mean(dim=0)) / X.std(dim=0), 0.)

# 学習データの標準化（ピクセル値を標準化）
# for i in range(X.shape[1]):
#     mean = X[:, i].mean()
#     std = X[:, i].std()
#     if(std == 0.):
#         X[:, i] = 0.
#     else:
#         X[:, i] = (X[:, i] - mean) / std


コメントアウトしている部分のようにfor文でもできるが、  
上記のようにdim=0を指定して各列ごとに平均、標準偏差を求めた方が簡単。  
ゼロ割はnan_to_num(tensor, 0)で変えておけばよい。  

In [28]:
# 重みW^T、バイアス項bの初期化
W = torch.rand(size=(10, 64) ,requires_grad=True) #出力×入力
b = torch.rand(size=(1, 10), requires_grad=True) # 1 x 出力

In [7]:
# softmax関数の実装
def softmax_func(X):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    '''
    # x_kが大きすぎると、e^xがinfになるのでmax(x_1, x_2,...,x_K)を各x_kから引く。
    # 各データ（各行）について最大値を求める必要があるので、dimにRank1(列方向で比較)を指定する。
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()

    # softmax関数の分母の計算
    # 各データについて合計したいので、dim=1を設定。また、分母が0になることを防ぐために分母の式に1e-10を足しておく
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10

    return e_x / denominator


In [8]:
# 多クラス分類に対応した交差エントロピー
def cross_entropy(y_true, y_pred):
    '''
    y_true: tensor。One-Hot Encoding済みの正解ラベル。
    y_pred: tensor。予測値。softmax関数の出力(0~100%)。
    '''
    # 損失を計算。最終的な損失はスカラーなので、dimを指定する必要はない。
    return - (y_true * torch.log(y_pred + 1e-10)).sum() / y_true.shape[0]


# ●ミニバッチ勾配降下法による学習
実装の方針：
* 毎回のepochの開始時に学習データをシャッフルし，毎回のepochで異なるミニバッチ群を作成
    * Xとyで別々にシャッフルすると、特徴量と目的変数の組み合わせが変わってしまうので、  
    Xかyのindexをシャッフルしてそこからミニバッチサイズ分取り出すという実装にした方が楽。  
    (もちろんX, yは初期状態でindexが対応している前提)
* 各バッチ毎の損失の平均を累積し， epochの最後にそのepochでの損失の平均を計算する(1データの平均損失を求める)
* バッチサイズには32(2^5)を指定 

## 実装前の実験

In [29]:
# 補足
shuffled_indices = np.random.permutation(len(target))
shuffled_indices

array([ 953, 1183, 1662, ..., 1149,  612,  497])

np.random.permutation(整数)とすると、np.arange(整数)の結果をシャッフルして出力する。  
リストの場合はそのままリストの要素がシャッフルされる。  
なので、上記のように目的変数の数（＝データの数）を引数に入れれば、  
index=0～データ数-1までのインデックスがシャッフルされた形で取得できる。  

In [30]:
# ミニバッチのサイズ定義
batch_size = 2**5
# 全ミニバッチの数。ミニバッチサイズで割ったときの余りも考慮してプラス１
batch_num = len(target) // batch_size + 1
# 最後のミニバッチのデータ数
batch_remainder_num = len(target) % batch_size

start = 0
end = batch_size

for i in range(3):
    print(f'start idx: {start}, end idx: {end}')
    print(shuffled_indices[start : end])
    start += batch_size

    if (end + batch_size) <= 96:
        end += batch_size
    else:
        end += batch_remainder_num

start idx: 0, end idx: 32
[ 953 1183 1662 1436  841 1527  174  830 1238 1515 1028 1142 1012  294
 1150  154  663  115 1065   37  932  370 1312  172 1417 1272  169  474
  484   73  605  393]
start idx: 32, end idx: 64
[1359 1153  785 1107 1757  776   26 1772 1226  540  866  842  797  438
  401 1102  304 1109 1597  965 1245  446 1159  652  656 1450  187 1618
  988 1510  989 1574]
start idx: 64, end idx: 96
[ 516  674  102 1437  606 1068  394 1268 1485 1103  379 1055 1025   83
  495 1760  481 1579 1743  826  959  916 1498   50  994 1528  193  575
 1472 1449 1707 1302]


## 実装

In [33]:
# ミニバッチのサイズ定義
batch_size = 32

# 全ミニバッチの数。ミニバッチサイズで割ったときの余りも考慮してプラス１
batch_num = len(target) // batch_size + 1

# 最後のミニバッチのデータ数
batch_remainder_num = len(target) % batch_size


# for文で学習ループ作成
for epoch in range(5):
    # epochごとの損失を蓄積する用の変数
    running_loss = 0

    # バッチごとの処理対象データ開始・終了インデックスを初期化
    batch_start_idx = 0
    batch_end_idx = batch_size

    # シャッフル後のindex
    shuffled_indices = np.random.permutation(len(target))

    # ミニバッチ勾配降下法
    for _ in range(batch_num):
        #print(f'batch{i}: start idx:{batch_start_idx}, end idx:{batch_end_idx}')
        # シャッフル後のindexからy,Xで同じ範囲を取り出しだしてミニバッチ作成
        batch_indices = shuffled_indices[batch_start_idx : batch_end_idx]
        y_true_batch = y_true[batch_indices, :]
        X_batch = X[batch_indices, :]
        #pdb.set_trace()

        # zの計算
        z = X_batch @ W.T + b # 1 x クラス数

        # softmaxで予測値算出
        y_pred = softmax_func(z)

        # 損失(L)計算.lossはtensorなので.item()で値だけ取り出す
        loss = cross_entropy(y_true_batch, y_pred)
        loss_log.append(loss.item())
        # 計算したlossを累積
        running_loss += loss.item()

        # Lの勾配計算。これをすることでw,bによるlossの偏微分係数が求められるようになる
        loss.backward()

        # パラメタ更新。更新するだけなので勾配の保持は不要。
        with torch.no_grad():
            W -= learning_rate * W.grad
            b -= learning_rate * b.grad

        # 勾配初期化
        W.grad.zero_()
        b.grad.zero_()

        # batch開始・終了インデックスを更新
        batch_start_idx += batch_size

        if (batch_end_idx + batch_size) <= len(y_true):
            batch_end_idx += batch_size
        else:
            # もしbatchの終了インデックスがデータ数を超えてしまったら（最後のbatchに到達したら）、
            # 残りのデータ数分だけ終了インデックスをずらす
            batch_end_idx += batch_remainder_num

    # epochの最終的な損失を出力。各バッチの損失の累積を全バッチ数で割って平均を求める
    print(f'Loss of epoch({epoch}): {running_loss / batch_num}')

Loss of epoch(0): 2.278509156745777
Loss of epoch(1): 1.0543543665032638
Loss of epoch(2): 0.7077103074182544
Loss of epoch(3): 0.5572720235377028
Loss of epoch(4): 0.47082332508605823


### ※補足 ミニバッチサイズで割り切れなかった余りのデータについて
ミニバッチサイズで割り切れなかった余りのデータを取り扱うために  
endのインデックスについて工夫をしたが、実はこれはスライシングの仕様上不要である。  
何故ならば下記の通り、スライシングの終点のインデックスが配列の最大インデックスを超えていても  
存在する範囲まで取得して返してくれるため。(少し気持ち悪いが実装が楽になるので有効活用しよう)  

In [46]:
print(shuffled_indices[0:])
print(shuffled_indices[0:2000000000000000000000])

[ 611 1518  808 ... 1271  860  459]
[ 611 1518  808 ... 1271  860  459]


### ※補足 pdb(python debuger)モジュールについて
pdbはデバッグのためのモジュール。  
pdb.set_trace()を差し込んだ箇所までコードが実行（ブレイクポイントの役割）され、  
ipyの形でインタラクティブに変数の中身などを確認することが出来て非常に便利。  
抜けるときはexitを入力する。  

# ●予測結果を見てみる
とりあえず学習に使用したXを使用して、どのような結果になっているか見てみる。  
(当然、学習データをインプットとするので精度的には過学習気味になる)

In [34]:
# 結果確認用の特徴量を定義
X_val = X

# 学習後のW,bを使って線形変換の式を計算
Z_val = X_val @ W.T + b

# 予測確率を出力
y_pred_proba_val = softmax_func(Z_val)

In [35]:
y_pred_proba_val[0]

tensor([9.6086e-01, 2.6243e-05, 2.8885e-03, 1.1751e-04, 8.5666e-03, 3.9422e-04,
        2.7395e-03, 7.4099e-03, 3.7360e-04, 1.6619e-02],
       grad_fn=<SelectBackward0>)

ソフトマックス関数により確率はわかるが、最終的にどのクラス（0～9）の判定となったかを知りたい場合もある。  
このようなときはtorch.argmax(dim=1)で各行のどのインデックスが最大値をとるのかを確認すればよい。  

In [36]:
torch.argmax(y_pred_proba_val, dim=1)

tensor([0, 1, 2,  ..., 8, 9, 8])

In [37]:
# one hot後の正解ラベルも同様
y_true.argmax(dim=1)

tensor([0, 1, 2,  ..., 8, 9, 8])

In [39]:
# accuracyを計算。==をとることにより、True or Falseが返される。
# pythonではTrueは1,Falseは0の扱いなので、合計をとることで正解数がカウントできる。
(torch.argmax(y_pred_proba_val, dim=1) == y_true.argmax(dim=1)).sum() / len(y_true)

tensor(0.8943)

学習データを検証データとして使用したため、当然高い値になる。  
少なくとも全く頓珍漢な予測値にはなっていなさそうということはわかる。  
ちゃんとした精度の検証は別途。